# Universal Sentence Encoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

# plotting
import matplotlib.pyplot as plt

import os
import re
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# scikit-learn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

In [ ]:
model_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(model_url)

In [ ]:
df = pd.read_csv('drive/MyDrive/MetacriticPCGamesofAllTime.csv')
# df = df.loc[:, ['title', 'critic_reviews']]
df = df.dropna(subset='critic_reviews')
df = df.reset_index(drop=True)

In [ ]:
df.head()

,title,publisher,release_date,metascore,user_score,metascore_count,user_rating_count,platform,genres,summary,critic_reviews
0,Batman: Arkham City,Warner Bros. Interactive Entertainment,"Nov 21, 2011",91.0,8.7,27,2970 Ratings,PC,"['Action Adventure', 'Fantasy', 'Fantasy', 'Op...",DC Comics' Dark Knight clashes with the Joker ...,Batman: Arkham City is one of those games that...
1,Decisive Battles of WWII Vol 2: Korsun Pocket,Matrix Games,"Jul 31, 2003",89.0,6.8,6,25 Ratings,PC,"['Strategy', 'Turn-Based', 'Wargame', 'Turn-Ba...","Korsun Pocket is the long awaited sequel to ""T...","Somehow, Korsun Pocket manages to capture ever..."
2,Night in the Woods,Finji,"Feb 21, 2017",88.0,7.7,30,422 Ratings,PC,"['Fantasy', 'Action Adventure', 'Adventure', '...",Night In The Woods is a 2D story-focused adven...,"A warm, moving tale of alienation and redempti..."
3,Fallout 2,Interplay,"Oct 29, 1998",86.0,9.1,15,1640 Ratings,PC,"['Role-Playing', 'First-Person', 'First-Person...",It's been 80 long years since your ancestor tr...,"Nothing short of superb - it takes ""Fallout,"" ..."
4,Titanfall 2,Electronic Arts,"Oct 28, 2016",86.0,8.6,30,1659 Ratings,PC,"['Action', 'Shooter', 'First-Person', 'Arcade']","In single player, Titanfall 2 delivers a craft...","Titanfall 2 goes for feel above all else, and ..."


In [ ]:
df.critic_reviews = df.critic_reviews.apply(lambda x: eval(x))
df.critic_reviews = df.critic_reviews.apply(lambda x: "".join(x))

In [ ]:
df.loc[1,"summary"]

'Korsun Pocket is the long awaited sequel to "The Ardennes Offensive" in the Decisive Battles system. The game uses a greatly improved version of the Ardennes Offensive engine to recreate the desperate German attempt to escape encirclement on the Russian Front early in 1944. The battle is a tense and exciting struggle, with neither side having a decisive advantage, as the Russians struggle to form the pocket, then try to resist successive German rescue efforts and last ditch attempts at breakout. [SSG]'

In [ ]:
critic_scores = model(df.critic_reviews)

In [ ]:
critic_scores[0]

<tf.Tensor: shape=(512,), dtype=float32, numpy=
array([-0.00378745, -0.05950264, -0.01989486,  0.01461046,  0.05734235,
       -0.05496167,  0.0388711 , -0.03803613,  0.06134276, -0.02672354,
        0.0622929 , -0.02621713, -0.03707981, -0.00148052,  0.01797827,
        0.05565865,  0.05767933,  0.01653616,  0.00069068,  0.06037844,
       -0.0265757 , -0.03778035,  0.03111732,  0.04110816,  0.06132976,
       -0.06104199,  0.04477378,  0.05638934, -0.00277537,  0.01690362,
        0.01369658,  0.0613363 , -0.04151008,  0.06201003,  0.0337183 ,
       -0.04711356,  0.05608914,  0.01597372, -0.05746311,  0.02778942,
        0.00210813, -0.01584406, -0.04585347, -0.05358534, -0.00141755,
       -0.05884661, -0.0247856 , -0.05509973, -0.06094493,  0.060739  ,
        0.06230404, -0.03618051, -0.00126048,  0.01120688, -0.04287052,
        0.05797909, -0.04800941, -0.06050495,  0.04705943,  0.02891401,
        0.06141119, -0.05621429,  0.01161974,  0.04740025,  0.05460729,
        0.059306

In [ ]:
pca = PCA(n_components=3)
data_pca = pca.fit_transform(critic_scores)

In [ ]:
data_pca

array([[ 0.01462388,  0.1090354 ,  0.21131594],
       [ 0.10487826, -0.05334549, -0.10381599],
       [-0.32867518,  0.09486055, -0.03852454],
       ...,
       [ 0.08988356, -0.14308043,  0.16564241],
       [ 0.16412392, -0.08057128,  0.16354341],
       [-0.08426017, -0.0529163 , -0.05981948]])

In [ ]:
# Extract PCA components
x_pca = data_pca[:, 0]
y_pca = data_pca[:, 1]
z_pca = data_pca[:, 2]

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=x_pca,
    y=y_pca,
    z=z_pca,
    mode='markers',
    marker=dict(
        size=8,
        color=z_pca,
        colorscale='Viridis',
        opacity=0.8,
    )
)])

fig.add_trace(go.Scatter3d(
    x=[0],
    y=[0],
    z=[0],
    mode='markers',
    marker=dict(
        size=12,
        color='red',
        opacity=1.0,
    ),
    name='Desired Point'
))


# Set axis labels
fig.update_layout(scene=dict(xaxis_title='PC1', yaxis_title='PC2', zaxis_title='PC3'))

# Show the plot
fig.show()

In [ ]:
critic_scores

<tf.Tensor: shape=(5392, 512), dtype=float32, numpy=
array([[-0.00378745, -0.05950264, -0.01989486, ...,  0.05222674,
         0.06230377,  0.03081096],
       [-0.05968456, -0.05016458,  0.02498764, ..., -0.05401561,
         0.04629549, -0.04519242],
       [-0.04050203, -0.03866656,  0.0619152 , ...,  0.00714321,
         0.07319137,  0.06363265],
       ...,
       [-0.01160222, -0.06679604,  0.01070888, ..., -0.02746653,
         0.07780056,  0.00386202],
       [-0.00425243, -0.03797514, -0.03834215, ...,  0.00407098,
         0.05818084,  0.04903199],
       [ 0.00387745, -0.07395802, -0.05897219, ...,  0.06298765,
         0.07565371,  0.03853623]], dtype=float32)>

In [ ]:
nn = NearestNeighbors(n_neighbors=10)
nn.fit(critic_scores)

NearestNeighbors(n_neighbors=10)

In [ ]:
user_text = "good level design"
user_text = [user_text]
user_text_scores = model(user_text)
recommendations = nn.kneighbors(user_text_scores, return_distance=False)
print(recommendations)
df.iloc[recommendations[0],:].sort_values(by='metascore', ascending=False)

[[2059 4049 3675 4622 2743  226 4051 3939 3937 3772]]


,title,publisher,release_date,metascore,user_score,metascore_count,user_rating_count,platform,genres,summary,critic_reviews
226,Yooka-Laylee and the Impossible Lair,Team17,"Oct 8, 2019",81.0,6.4,13,68 Ratings,PC,"['Action', 'Platformer', '2D']",Yooka & Laylee are back in a brand-new platfor...,Playtonic has absolutely smashed it with Yooka...
2743,So Many Me,Extend Interactive,"Jul 17, 2014",78.0,7.1,7,22 Ratings,PC,"['Miscellaneous', 'Action', 'Puzzle', 'Puzzle'...","Control Dave and his many clones to explore, s...",A wonderful game. Very well designed and highl...
2059,Airscape: The Fall of Gravity,Cross-Product,"Aug 11, 2015",75.0,6.6,9,17 Ratings,PC,"['Action', 'Platformer', '2D']",Airscape: The Fall of gravity is a gravity-shi...,Airscape: The Fall of Gravity is an excellentl...
4622,Blood II: The Chosen,GT Interactive,"Oct 31, 1998",72.0,7.1,14,71 Ratings,PC,"['Action', 'Shooter', 'Shooter', 'First-Person...","Take control of any one of ""The Chosen"" -- fou...","Simply put, Blood II is more of the same: more..."
4049,YesterMorrow,Blowfish Studios,"Nov 5, 2020",71.0,tbd,5,NaN,PC,"['Action', 'Action Adventure', 'General', 'Pla...",YesterMorrow is a single-player time travellin...,"YesterMorrow is a thoroughly enjoyable, challe..."
4051,B.I.O.T.A.,Retrovibe,"Apr 12, 2022",71.0,tbd,7,NaN,PC,"['Action', 'Platformer', '2D', 'Metroidvania']",B.I.O.T.A. is a 2D metroidvania action-platfor...,I honestly wasn’t sure what to expect from B.I...
3675,FOX n FORESTS,EuroVideo Medien,"May 17, 2018",69.0,7.4,7,13 Ratings,PC,"['Action', 'Platformer', '2D']",Fox n Forests is a 2D action platformer with R...,The core platforming and action-oriented comba...
3939,Chrome,Strategy First,"Oct 28, 2003",69.0,6.8,21,61 Ratings,PC,"['Action', 'Shooter', 'Shooter', 'First-Person...","Take on the role of Logan, a veteran mercenary...",The type of game that you know you should find...
3937,Light Fall,Bishop Games,"Apr 26, 2018",69.0,6.8,10,6 Ratings,PC,"['Action', 'Platformer', '2D']",Explore the Forgotten World of Numbra to uncov...,Light Fall is a simple platformer designed gre...
3772,Inside My Radio,Iceberg Interactive,"May 11, 2015",68.0,7.4,8,8 Ratings,PC,"['Action', 'Platformer', '2D']",Can you beat the beat? Journey as a green LED ...,Inside my Radio is a great platformer that des...
